In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')
print(train.shape, test.shape)

(1460, 81) (1459, 80)


In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
sample_submission = pd.read_csv('/kaggle/input/home-data-for-ml-course/sample_submission.csv')
display(sample_submission)

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
...,...,...
1454,2915,167081.220949
1455,2916,164788.778231
1456,2917,219222.423400
1457,2918,184924.279659


In [5]:
!pip install pycaret --ignore-installed llvmlite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 14.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 20.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.2/435.2 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 23.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.9/25.9 MB 17.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 3.6 MB/s eta 0:00:00
  Preparing 

In [7]:
from pycaret.regression import *

In [14]:
reg = setup(data = train, target = 'SalePrice', session_id = 100, 
              transformation = True, 
              transform_target = True,
              remove_multicollinearity = True,
              multicollinearity_threshold = 0.85,
              remove_outliers = True,
              handle_unknown_categorical = True,
              ignore_low_variance = True,
              create_clusters = True,
              feature_selection = True,
              feature_selection_threshold = 0.7,
              log_experiment = True,
              fold = 10,
              n_jobs = -1,
              use_gpu = True,
              pca = True,
              pca_components = 0.95,
              silent = True,
              ignore_features = ['Id'],
              categorical_features = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'])

,Description,Value
0,session_id,100
1,Target,SalePrice
2,Original Data,"(1460, 81)"
3,Missing Values,True
4,Numeric Features,18
5,Categorical Features,61
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(970, 98)"


PicklingError: Can't pickle <class 'pyod.models.pca.PCA'>: it's not found as pyod.models.pca.PCA

In [13]:
train.select_dtypes(['object']).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [15]:
k = compare_models(sort = 'MAE', n_select = 3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,17373.1538,891414433.7972,28485.5658,0.8464,0.1283,0.0963,0.1610
br,Bayesian Ridge,17754.4206,975727422.6707,29569.6251,0.8353,0.1292,0.0966,0.0820
ridge,Ridge Regression,18018.9547,984724926.1652,29806.4473,0.8326,0.1304,0.0980,0.0620
lar,Least Angle Regression,18068.2014,987105645.0699,29858.8149,0.8321,0.1307,0.0983,0.1090
lr,Linear Regression,18068.2365,987110316.0740,29858.8934,0.8321,0.1307,0.0983,0.0700
catboost,CatBoost Regressor,20246.1489,1244268893.0962,34092.1398,0.8068,0.1498,0.1074,22.6960
omp,Orthogonal Matching Pursuit,20591.2736,1335028883.8696,34651.4367,0.7773,0.1545,0.1138,0.0380
et,Extra Trees Regressor,21471.1723,1313031796.0105,35256.7588,0.7934,0.1590,0.1155,1.5220
lightgbm,Light Gradient Boosting Machine,21522.4281,1230433896.4908,34195.5504,0.8047,0.1587,0.1178,0.8980
gbr,Gradient Boosting Regressor,21856.2549,1254611062.4899,34526.4024,0.7997,0.1607,0.1189,2.1050


In [16]:
huber_tuned = tune_model(k[0])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,15862.5918,415825679.7259,20391.8042,0.9310,0.1246,0.0983
1,17145.1018,722624362.0816,26881.6733,0.8361,0.1225,0.0925
2,17475.2917,522369250.6816,22855.3987,0.9177,0.1167,0.0945
3,17316.3996,833399984.9462,28868.6679,0.9049,0.1264,0.0938
4,15826.6935,694826031.3899,26359.5529,0.8838,0.1222,0.0906
5,21182.3434,2718865640.3103,52142.7429,0.4728,0.1829,0.1258
6,18875.4983,658717236.8845,25665.4873,0.8696,0.1129,0.0944
7,16598.4344,747390164.7188,27338.4375,0.8624,0.1229,0.0898
8,18821.0222,1233587476.1848,35122.4640,0.8764,0.1173,0.0881


In [19]:
evaluate_model(huber_tuned)

,Parameters
alpha,0.7
epsilon,1.55
fit_intercept,True
max_iter,100
tol,1e-05
warm_start,False
power_transformer_method,box-cox
power_transformer_standardize,True
regressor,"HuberRegressor(alpha=0.7, epsilon=1.55, fit_in..."


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [20]:
predict_model(huber_tuned)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Huber Regressor,17900.4776,833513233.2968,28870.6293,0.8705,0.1589,0.1092


,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,Component_9,Component_10,...,Component_91,Component_92,Component_93,Component_94,Component_95,Component_96,Component_97,Component_98,SalePrice,Label
0,-2.082881,0.971040,0.199074,-0.977393,-0.021433,-1.018720,-0.245461,0.844727,-0.797186,0.330398,...,-0.302768,-0.067937,-0.045270,-0.087808,-0.383123,0.228040,0.025102,-0.049565,120500,125137.100922
1,2.301972,-2.440907,0.234219,0.620178,-1.033239,-1.728452,0.868020,1.169640,0.295631,-0.159258,...,0.258658,0.108830,-0.113172,-0.141225,-0.034470,0.103173,0.019602,0.040024,196500,187203.112373
2,1.648532,-0.498249,1.435873,-0.422487,0.183081,1.199276,0.368388,1.286007,1.310398,0.725740,...,-0.073270,-0.041324,-0.004485,-0.135110,0.232401,-0.086337,0.194777,-0.069176,176000,173246.709547
3,2.891373,-2.319316,-0.046901,0.361521,-1.622178,-0.095738,-0.017120,1.100565,0.369270,0.166155,...,-0.078243,0.121320,-0.015982,0.013379,0.024860,0.105154,0.025965,-0.101303,213500,201778.471299
4,-1.245462,1.531346,0.219750,-0.073024,-0.632952,0.689069,-1.297209,0.602854,0.963794,0.264193,...,-0.101450,-0.065345,-0.312318,-0.828960,0.121429,0.222591,-0.047615,0.222551,171000,145910.274673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,-4.233364,-0.758784,2.242933,-0.254304,0.244949,0.418382,-0.318474,-0.957970,-0.698096,0.590027,...,0.550662,0.134379,0.061193,-0.360207,0.227749,0.132592,-0.088785,0.247540,113000,110241.527445
435,2.620063,-0.810811,-1.440652,-0.735582,1.034366,-1.325171,0.653498,-0.744117,0.077443,-0.641357,...,-0.045799,0.261528,-0.125451,-0.125608,-0.107412,0.005246,-0.151907,0.174983,241000,198217.625819
436,0.897778,1.148961,-2.469693,0.986144,0.257655,0.200083,1.479778,0.520940,-2.204275,-1.113870,...,0.374291,0.375239,0.330374,0.496733,0.035752,-0.237304,0.161614,0.130330,229000,238627.910274
437,1.606091,-0.724387,-2.622305,0.294938,-0.830445,-0.912504,-0.848749,0.444601,1.118266,0.630210,...,0.065576,-0.021599,-0.124416,0.137817,-0.117144,0.122614,0.029991,-0.060561,185000,197405.293422


In [21]:
final_huber = finalize_model(huber_tuned)

In [22]:
preds = predict_model(final_huber, data = test)

In [23]:
preds

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Label
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,124468.075623
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,160212.459908
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,190765.941847
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,207239.863653
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,166718.927307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,90105.207397
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,98461.150150
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,191736.057114
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,116737.187453


In [24]:
submit = pd.DataFrame()
submit['Id'] = preds['Id']
submit['SalePrice'] = preds['Label']
submit.to_csv('submission.csv', index = False)